In [2]:
import numpy as np

good_patterns = [[1, 0, 0, 1]]
training_data_length = 1000

def getTrainingData(length, pattern_length):
    training_data = np.array([np.array([0, 1]).repeat(pattern_length / 2)] * length)
    
    for pattern in training_data:
        np.random.shuffle(pattern)   
    
    return training_data

def getCorrectResults(patterns):
    return np.array([1 if list(pattern) in good_patterns else 0 for pattern in patterns])

training_data = getTrainingData(training_data_length, pattern_length=4)
correct_results_training_data = getCorrectResults(training_data)

In [3]:
import tensorflow.keras as keras


NN = keras.Sequential()
NN.add(keras.Input(shape=(4,)))
NN.add(keras.layers.Dense(1, activation="sigmoid")) # sigmoid(x) = 1 / (1 + exp(-x)) here

NN.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

NN.fit(training_data, correct_results_training_data, epochs=150)

NN.summary()

weights = NN.get_weights()

# print the weights
for i, layer_weights in enumerate(weights):
    print(f"Layer {i}:")
    for j, weight in enumerate(layer_weights):
        print(f"Weight {j}: {weight}")

Epoch 1/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - accuracy: 0.5229 - loss: 0.7012 
Epoch 2/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step - accuracy: 0.5200 - loss: 0.6690
Epoch 3/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step - accuracy: 0.6542 - loss: 0.6306
Epoch 4/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - accuracy: 0.6565 - loss: 0.6166
Epoch 5/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - accuracy: 0.6832 - loss: 0.5738
Epoch 6/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - accuracy: 0.6781 - loss: 0.5497
Epoch 7/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step - accuracy: 0.6838 - loss: 0.5195
Epoch 8/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.8366 - loss: 0.4990
Epoch 9/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step - accuracy: 0.8368 - loss: 0.4759
Epoch 10/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - accuracy: 0.8573 - loss: 0.4586
Epoch 11/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step - accuracy: 1.0000 - loss: 0.4446
Epoch 12/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17 (72.00 B)

 Trainable params: 5 (20.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12 (52.00 B)

Layer 0:
Weight 0: [1.6897647]
Weight 1: [-2.8278818]
Weight 2: [-3.2880359]
Weight 3: [1.7159966]
Layer 1:
Weight 0: -1.605256199836731


Now we want to test the Network with a  new list of patterns.

In [4]:
test_data = np.copy(training_data)
np.random.shuffle(test_data)

# NN.predict returns an array with shape=(trainging_data_length = 1000, output_neurons = 1) 
# so we flatten it to get a 1D array with our predictions
results_on_test_data = NN.predict(test_data).flatten() 
correct_results_test_data = getCorrectResults(test_data)

all_correct = np.array_equal(correct_results_test_data, np.round(results_on_test_data))
max_difference = max(abs(results_on_test_data - correct_results_test_data))

print(f"all test data correct, when rounded: {all_correct}")
print(f"maximum difference from correct result: {max_difference}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
all test data correct, when rounded: True
maximum difference from correct result: 0.14178961515426636


In [5]:
good_patterns = [
    list(np.array(list(zip(np.ones(32), np.zeros(32)))).flatten()),
    list(np.array(list(zip(np.zeros(32), np.ones(32)))).flatten())
]

training_data_length = 50_000

def getTrainingData(length, pattern_length):
    training_data = np.array([np.array([0, 1]).repeat(pattern_length / 2)] * length)
    
    for i, pattern in enumerate(training_data):
        # this time we additionally inject some good patterns, 
        # because otherwise there wouldnt be enough good patterns in training_data
        rand = np.random.rand()
        if rand < 0.2:
            training_data[i] = np.array(good_patterns[0], dtype=int)
        elif rand < 0.4:
            training_data[i] = np.array(good_patterns[0], dtype=int)
        else: 
            np.random.shuffle(pattern) # shuffle in place, pattern is still the actual element from the training_data array
    
    return training_data

training_data = getTrainingData(training_data_length, pattern_length=64)
correct_results_training_data = getCorrectResults(training_data)

In [12]:
NN = keras.Sequential()
NN.add(keras.Input(shape=(64,)))
NN.add(keras.layers.Dense(32, activation="relu"))
NN.add(keras.layers.Dense(16, activation="relu"))
NN.add(keras.layers.Dense(1, activation="sigmoid")) # sigmoid(x) = 1 / (1 + exp(-x)) here

NN.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

NN.fit(training_data, correct_results_training_data, epochs=15)

NN.summary()

Epoch 1/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 628us/step - accuracy: 0.9830 - loss: 0.0661
Epoch 2/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 605us/step - accuracy: 1.0000 - loss: 4.7008e-05
Epoch 3/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 603us/step - accuracy: 1.0000 - loss: 7.0326e-06
Epoch 4/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 625us/step - accuracy: 1.0000 - loss: 2.4152e-06
Epoch 5/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 604us/step - accuracy: 1.0000 - loss: 9.5256e-07
Epoch 6/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 596us/step - accuracy: 1.0000 - loss: 3.2807e-07
Epoch 7/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 614us/step - accuracy: 1.0000 - loss: 1.3790e-07
Epoch 8/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 629us/step - accuracy: 1.0000 - loss: 7.2187e-08
Epoch 9/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 604us/step - accuracy: 1.0000 - loss: 2.6888e-08
Epoch 10/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 589us/step - accuracy: 1.0000 - loss: 1.4195e-08
Epoch 11/35
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1s 596us/step - accu

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,877 (30.77 KB)

 Trainable params: 2,625 (10.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,252 (20.52 KB)

In [13]:
test_data = getTrainingData(500_000, pattern_length=64)

results_on_test_data = NN.predict(test_data).flatten()
correct_results_test_data = getCorrectResults(test_data)

all_correct = np.array_equal(correct_results_test_data, np.round(results_on_test_data))
max_difference = max(abs(results_on_test_data - correct_results_test_data))

print(f"all test data correct, when rounded: {all_correct}")
print(f"maximum difference from correct result: {max_difference}")

15625/15625 ━━━━━━━━━━━━━━━━━━━━ 7s 428us/step
all test data correct, when rounded: True
maximum difference from correct result: 0.00798442866653204
